In [7]:
%pip install geojson_pydantic


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import requests
import pandas as pd
import geopandas as gpd
from lonboard import viz
import h3

In [5]:
BASE_URL = "http://localhost:8000"
FIELDS_ENDPOINT = f"{BASE_URL}/fields"
SUMMARY_ENDPOINT = f"{BASE_URL}/summary"

In [6]:
response = requests.get(FIELDS_ENDPOINT)
if response.status_code != 200:
    raise Exception(f"Failed to get fields: {response.text}")

available_fields = response.json()
print("Available Fields:", available_fields)

Available Fields: ['__index_level_0__', 'ogc_fid', 'sum_pop_f_0_2020', 'sum_pop_f_10_2020', 'sum_pop_f_15_2020', 'sum_pop_f_1_2020', 'sum_pop_f_20_2020', 'sum_pop_f_25_2020', 'sum_pop_f_30_2020', 'sum_pop_f_35_2020', 'sum_pop_f_40_2020', 'sum_pop_f_45_2020', 'sum_pop_f_50_2020', 'sum_pop_f_55_2020', 'sum_pop_f_5_2020', 'sum_pop_f_60_2020', 'sum_pop_f_65_2020', 'sum_pop_f_70_2020', 'sum_pop_f_75_2020', 'sum_pop_f_80_2020', 'sum_pop_m_0_2020', 'sum_pop_m_10_2020', 'sum_pop_m_15_2020', 'sum_pop_m_1_2020', 'sum_pop_m_20_2020', 'sum_pop_m_25_2020', 'sum_pop_m_30_2020', 'sum_pop_m_35_2020', 'sum_pop_m_40_2020', 'sum_pop_m_45_2020', 'sum_pop_m_50_2020', 'sum_pop_m_55_2020', 'sum_pop_m_5_2020', 'sum_pop_m_60_2020', 'sum_pop_m_65_2020', 'sum_pop_m_70_2020', 'sum_pop_m_75_2020', 'sum_pop_m_80_2020']


In [7]:
from typing import List, Dict, Any, Literal

from geojson_pydantic import Feature, Polygon

AOIModel = Feature[Polygon, Dict]

aoi = {
    "type": "Feature",
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [-74.1, 40.6],
                [-73.9, 40.6],
                [-73.9, 40.8],
                [-74.1, 40.8],
                [-74.1, 40.6]
            ]
        ]
    },
    "properties": {}
}

feat = AOIModel(**aoi)

In [43]:
aoi = {
    "type": "Feature",
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [-74.1, 40.6],
                [-73.9, 40.6],
                [-73.9, 40.8],
                [-74.1, 40.8],
                [-74.1, 40.6]
            ]
        ]
    },
    "properties": {}

}

# Define the Request Payload
request_payload = {
    "aoi": aoi,
    "spatial_join_method": "centroid",
    "fields": available_fields  # Use all available fields
}

# Get Summary Data
response = requests.post(SUMMARY_ENDPOINT, json=request_payload)
if response.status_code != 200:
    raise Exception(f"Failed to get summary: {response.text}")

summary_data = response.json()
print("Summary Data:", summary_data)

# Convert Summary Data to DataFrame
summary_df = pd.DataFrame(summary_data)

Summary Data: [{'hex_id': '862a1008fffffff', '__index_level_0__': 2436230, 'ogc_fid': 11, 'sum_pop_f_0_2020': 3355.96850585938, 'sum_pop_f_10_2020': 12371.3955078125, 'sum_pop_f_15_2020': 15563.8896484375, 'sum_pop_f_1_2020': 12494.43359375, 'sum_pop_f_20_2020': 30224.130859375, 'sum_pop_f_25_2020': 42427.28125, 'sum_pop_f_30_2020': 34711.5625, 'sum_pop_f_35_2020': 25574.31640625, 'sum_pop_f_40_2020': 20973.458984375, 'sum_pop_f_45_2020': 18116.025390625, 'sum_pop_f_50_2020': 18691.546875, 'sum_pop_f_55_2020': 21246.267578125, 'sum_pop_f_5_2020': 12426.166015625, 'sum_pop_f_60_2020': 22672.314453125, 'sum_pop_f_65_2020': 20404.287109375, 'sum_pop_f_70_2020': 17031.431640625, 'sum_pop_f_75_2020': 11438.015625, 'sum_pop_f_80_2020': 17598.7109375, 'sum_pop_m_0_2020': 3499.36499023438, 'sum_pop_m_10_2020': 12757.32421875, 'sum_pop_m_15_2020': 14690.669921875, 'sum_pop_m_1_2020': 13028.8857421875, 'sum_pop_m_20_2020': 24383.478515625, 'sum_pop_m_25_2020': 36570.5, 'sum_pop_m_30_2020': 33656

In [44]:
summary_df

,hex_id,__index_level_0__,ogc_fid,sum_pop_f_0_2020,sum_pop_f_10_2020,sum_pop_f_15_2020,sum_pop_f_1_2020,sum_pop_f_20_2020,sum_pop_f_25_2020,sum_pop_f_30_2020,...,sum_pop_m_40_2020,sum_pop_m_45_2020,sum_pop_m_50_2020,sum_pop_m_55_2020,sum_pop_m_5_2020,sum_pop_m_60_2020,sum_pop_m_65_2020,sum_pop_m_70_2020,sum_pop_m_75_2020,sum_pop_m_80_2020
0,862a1008fffffff,2436230,11,3355.968506,12371.395508,15563.889648,12494.433594,30224.130859,42427.281250,34711.562500,...,21449.726562,18393.367188,17531.789062,18519.333984,12790.003906,17991.046875,15532.092773,12730.666016,8303.866211,9250.683594
1,862a100d7ffffff,2436238,19,3169.981201,11957.441406,14855.185547,11801.993164,27791.644531,38212.468750,31441.304688,...,19612.835938,16889.306641,16264.783203,17167.789062,12373.455078,16715.095703,14265.351562,11801.837891,7603.230957,8574.225586
2,862a100dfffffff,2436239,20,5147.333008,21324.523438,22045.636719,19163.771484,27736.291016,35799.523438,33238.703125,...,20631.275391,18518.207031,18673.708984,19300.269531,21773.695312,18752.816406,14246.966797,12312.962891,7566.346680,8843.825195
3,862a10707ffffff,2436390,49,1031.586792,3744.323975,3917.221436,3840.647461,5425.830078,8483.667969,8123.698242,...,5050.240723,4325.686523,4091.522461,4049.447754,3862.147461,3625.974854,2958.136230,2380.195068,1527.440674,1581.647461
4,862a1070fffffff,2436391,50,285.267578,1234.033447,1272.867432,1062.065186,1501.954590,1971.187744,1970.330566,...,1456.805054,1328.027588,1317.732788,1337.024902,1246.012939,1250.812988,1011.778931,772.699219,493.368988,513.383545
5,862a10727ffffff,2436394,53,1815.303101,6547.267090,7142.462402,6758.461426,10860.016602,16565.375000,15229.623047,...,9380.048828,8014.259766,7586.525391,7627.609375,6769.341797,6963.209473,5769.195801,4668.747070,3006.267578,3161.916504
6,862a1072fffffff,2436395,54,1863.882080,7031.261719,8313.156250,6939.323730,14435.011719,20009.312500,16851.132812,...,10254.236328,8837.011719,8528.335938,8954.818359,7267.752441,8682.864258,7253.910156,6027.014648,3859.211914,4310.084961
7,862a10757ffffff,2436399,58,834.680481,3883.023193,3978.943359,3107.556152,4433.382324,5126.501953,5110.866699,...,3944.751953,3706.372559,3792.649170,3921.260742,3899.290527,3796.074463,2990.934570,2328.723389,1461.929077,1593.953613
8,862a10767ffffff,2436401,60,3148.868896,13023.148438,13454.269531,11723.392578,16926.832031,21846.343750,20248.339844,...,12466.598633,11185.371094,11283.837891,11677.320312,13300.945312,11366.205078,8611.406250,7453.043945,4575.960938,5342.289551
9,862a10777ffffff,2436403,62,3401.089600,14066.287109,14531.941406,12662.421875,18282.650391,23596.210938,21870.207031,...,13465.159180,12081.306641,12187.660156,12612.660156,14366.335938,12276.625000,9301.168945,8050.024902,4942.490234,5770.201172


In [46]:
import h3
from shapely.geometry import Polygon
import geopandas as gpd

def h3_to_polygon(h3_id):
    boundary = h3.h3_to_geo_boundary(h3_id, geo_json=True)
    print(f"h3_id: {h3_id}, boundary: {boundary}")
    return Polygon(boundary)

summary_df['geometry'] = summary_df['hex_id'].apply(h3_to_polygon)

gdf = gpd.GeoDataFrame(summary_df, geometry='geometry')

viz(gdf)

h3_id: 862a1008fffffff, boundary: ((-73.9609635556213, 40.829061732419916), (-74.00569135138367, 40.82168093780192), (-74.01944838354662, 40.79043914023696), (-73.98852232840895, 40.766594382212254), (-73.94384490497663, 40.77396440203852), (-73.93004320296495, 40.805189944379514), (-73.9609635556213, 40.829061732419916))
h3_id: 862a100d7ffffff, boundary: ((-73.94384490497663, 40.77396440203852), (-73.98852232840895, 40.766594382212254), (-74.0022744646159, 40.73537602621502), (-73.97139376102862, 40.71154393543933), (-73.92676660481357, 40.71890320501306), (-73.91296992347031, 40.75010530534533), (-73.94384490497663, 40.77396440203852))
h3_id: 862a100dfffffff, boundary: ((-73.92676660481357, 40.71890320501306), (-73.97139376102862, 40.71154393543933), (-73.98514098370819, 40.68034904926758), (-73.95430550947268, 40.656529678451555), (-73.90972851565844, 40.663878222244435), (-73.89593687206985, 40.69505685239637), (-73.92676660481357, 40.71890320501306))
h3_id: 862a10707ffffff, bounda

/Users/zacdez/Library/Caches/pypoetry/virtualenvs/notebooks-sNYx7QfP-py3.12/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…